In [1]:
import datasets
import numpy as np
from transformers import BertTokenizerFast, DataCollatorForTokenClassification, AutoModelForTokenClassification

c:\Users\kevin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = datasets.load_dataset("ktgiahieu/maccrobat2018_2020")

In [3]:
label_list = []

with open('./label_list.txt', 'r') as f:
    for line in f:
        parts = line.split(':')
        label = parts[1].strip()
        
        label_list.append(label)

In [4]:
dataset = dataset['train'].train_test_split(test_size=0.2)

In [5]:
validation_ds = dataset.pop('test')

In [6]:
dataset['validation'] = validation_ds

In [7]:
dataset.shape

{'train': (320, 2), 'validation': (80, 2)}

In [8]:
dataset['train']

Dataset({
    features: ['tokens', 'tags'],
    num_rows: 320
})

In [9]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [10]:
idx_to_label = {i: label for i, label in enumerate(label_list)}
label_to_idx = {label: i for i, label in enumerate(label_list)}

In [11]:
def tags_to_indices(example):
    example['tags'] = [label_to_idx[tag] for tag in example['tags']]
    return example

In [12]:
def indices_to_tags(example):
    example['tags'] = [idx_to_label[idx] for idx in example['tags']]
    return example

In [13]:
dataset = dataset.map(lambda x: tags_to_indices(x))

Map: 100%|██████████| 80/80 [00:00<00:00, 1631.60 examples/s]


In [14]:
def tokenize_and_align_labels(example, label_all_tokens=True):
    tokenized_input = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    labels = []

    for i, label in enumerate(example['tags']):
        word_ids = tokenized_input.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    
    tokenized_input["labels"] = labels
    return tokenized_input

In [15]:
q = tokenize_and_align_labels(dataset['train'][0:1])
len(q['input_ids'][0]), len(q['labels'][0])

(503, 503)

In [16]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 80/80 [00:00<00:00, 834.40 examples/s]


In [17]:
model = AutoModelForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    'test-ner',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=100,
    weight_decay=0.01,
)

In [19]:
metric = datasets.load_metric('seqeval')

C:\Users\kevin\AppData\Local\Temp\ipykernel_15952\520486469.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('seqeval')


In [20]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [21]:
import numpy as np
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score

def compute_metrics(p):
    logits, label_ids = p
    predictions = np.argmax(logits, axis=2)

    # Flatten the batches and remove the special tokens
    true_predictions = [
        label_list[p] for batch_preds, batch_labels in zip(predictions, label_ids) 
        for p, l in zip(batch_preds, batch_labels) if l != -100
    ]

    true_labels = [
        label_list[l] for batch_labels in label_ids
        for l in batch_labels if l != -100
    ]

    # Calculate metrics
    precision = precision_score([true_labels], [true_predictions])
    recall = recall_score([true_labels], [true_predictions])
    f1 = f1_score([true_labels], [true_predictions])
    accuracy = accuracy_score([true_labels], [true_predictions])

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy
    }


In [22]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [25]:
trainer.train()

model.save_pretrained('ner_model')

tokenizer.save_pretrained('tokenizer')

  0%|          | 0/1000 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
  1%|          | 10/1000 [00:07<11:58,  1.38it/s]c:\Users\kevin\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                 
  1%|          | 10/1000 [00:08<11:58,  1.38it/s]

{'eval_loss': 2.202951192855835, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.6222788267114834, 'eval_runtime': 0.7993, 'eval_samples_per_second': 100.084, 'eval_steps_per_second': 3.753, 'epoch': 1.0}


                                                 
  2%|▏         | 20/1000 [00:16<11:46,  1.39it/s]

{'eval_loss': 1.943030595779419, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.6222788267114834, 'eval_runtime': 0.7821, 'eval_samples_per_second': 102.295, 'eval_steps_per_second': 3.836, 'epoch': 2.0}


...

100%|██████████| 1000/1000 [13:09<00:00,  1.41it/s]

{'loss': 0.1388, 'learning_rate': 0.0, 'epoch': 100.0}


                                                   
100%|██████████| 1000/1000 [13:13<00:00,  1.26it/s]


{'eval_loss': 0.41415658593177795, 'eval_precision': 0.8161480808539632, 'eval_recall': 0.8753958587088916, 'eval_f1': 0.8447343676539726, 'eval_accuracy': 0.9232163020929072, 'eval_runtime': 1.0405, 'eval_samples_per_second': 76.888, 'eval_steps_per_second': 2.883, 'epoch': 100.0}
{'train_runtime': 793.0998, 'train_samples_per_second': 40.348, 'train_steps_per_second': 1.261, 'train_loss': 0.46105414581298826, 'epoch': 100.0}


('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.txt',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [23]:
import json

config = json.load(open('ner_model/config.json'))

In [24]:
config['id2label'] = idx_to_label
config['label2id'] = label_to_idx

json.dump(config, open('ner_model/config.json', 'w'))

model_fine_tuned = AutoModelForTokenClassification.from_pretrained('ner_model')

In [25]:
from transformers import pipeline

In [51]:
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)

example = "A 63-year-old woman with no known cardiac history presented with a sudden onset of dyspnea requiring intubation and ventilatory support out of hospital. She denied preceding symptoms of chest discomfort, palpitations, syncope or infection. The patient was afebrile and normotensive, with a sinus tachycardia of 140 beats/min."

ner_results = nlp(example)

for entry in ner_results:
    print(f"{entry['word']:_<20} {entry['entity']:_<50}({entry['score']}) ")

63__________________ B-Age_____________________________________________(0.971399188041687) 
-___________________ I-Age_____________________________________________(0.9865236878395081) 
year________________ I-Age_____________________________________________(0.9873430728912354) 
-___________________ I-Age_____________________________________________(0.9868648648262024) 
old_________________ I-Age_____________________________________________(0.9863035082817078) 
woman_______________ B-Sex_____________________________________________(0.9628715515136719) 
no__________________ B-History_________________________________________(0.7787923216819763) 
known_______________ I-History_________________________________________(0.9577584862709045) 
cardiac_____________ I-History_________________________________________(0.9156008958816528) 
history_____________ I-History_________________________________________(0.9439647793769836) 
d___________________ B-Sign_symptom____________________________________